## Examples of retrieving data from Scopus using Scopus API and Elsapy  
Elsapy: https://github.com/ElsevierDev/elsapy (Requres Python 3.x)  
Elsevier Scopus APIs https://dev.elsevier.com/sc_apis.html

In [1]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
from elsapy.elsprofile import ElsAuthor
from elsapy.elsdoc import AbsDoc

import json

### 1. Define some functions that we will use later

In [2]:
def affiliation_id_serch(schoolname, client):
    """Search affiliation ID by name
    client - object of the ElsClient class  """
    
    school_srch = ElsSearch(' AFFIL(%s)'%schoolname,'affiliation')
    school_srch.execute(client)
    return school_srch.results

def search_auth_by_name(fstname, lstname, schoolid, client):
    """Find author's Scopus ID by first, last name and affiliation ID.
    In case of an error, perform search without affiliation ID and return first author in the serach result
    """
    auth_srch = ElsSearch('AUTHLASTNAME(%s)'%lstname + ' AUTHFIRST(%s)'%fstname + ' AF-ID(%s)'%schoolid,'author')
    auth_srch.execute(client)

    #print ("auth_srch has", len(auth_srch.results), "results.")
    authorfound = auth_srch.results[0]
    
    if 'error' in authorfound.keys():
        status = 'error'
        auth_srch = ElsSearch('AUTHLASTNAME(%s)'%lstname + ' AUTHFIRST(%s)'%fstname ,'author')
        auth_srch.execute(client)

        #print ("auth_srch has", len(auth_srch.results), "results.")
        authorfound = auth_srch.results[0]
        if 'error' in authorfound.keys():
            status = 'error'
            return status, {}
        if 'affiliation-current' in authorfound.keys():
            if authorfound['affiliation-current']['affiliation-id']!=schoolid:
                status = 'new-affil'
            else:
                status = 'success'
        else:
            print(fstname, lstname, ' --- can not find affiliation-current in keys')
            status = 'warning'
    else:
        status = 'success'
    
    return status, authorfound

def auth_metrics(auth_id, client):
    """Read author metrics for a given Scopus author ID auth_id
    client - object of the ElsClient class 
    reurns status, author data, total number of paers, citations and h-index """
    
    my_auth = ElsAuthor(uri = 'https://api.elsevier.com/content/author/author_id/'+auth_id) 

    if my_auth.read(client):
        status = 'success'
    else:
        status = 'error'
        return status, None, None, None, None
    
    my_auth.read_metrics(client)

    if my_auth._data==None:
        npapers, ncitation, hindex = None, None, None
        status = 'error'
    else:
        status = 'success'
        
        npapers = my_auth._data['coredata']['document-count']
        ncitation = my_auth._data['coredata']['citation-count']
        hindex = my_auth._data['h-index']
        
    return status,my_auth._data, npapers, ncitation, hindex

def author_pubs(author_id, client):
    """
    Obtain publication list for a given Scopus author ID author_id
    client - object of the ElsClient class
    returns publication list and full output of the ElsSearch search request
    """
    doc_srch = ElsSearch('AU-ID(%s)'%author_id,'scopus')
    doc_srch.execute(client, get_all = True)

  
    pubs = []
    for rslt in doc_srch.results:
               
        year = rslt['prism:coverDate'].split('-')[0]
        citedby = rslt['citedby-count']
        scopusid = rslt['dc:identifier'].split(':')[1]
        title = rslt['dc:title']
        jrnlname = rslt['prism:publicationName']
        pubs.append({'year':year, 'cited':citedby, 'scopusid': scopusid, 'title': title, 'journal': jrnlname})
 
    return pubs, doc_srch.results


## 2. Initialize client (object of the ElsClient class) with your apikey

In [3]:
apikey = '27323cdf20b935eb21b35737c15553b6' # insert a valid apikey

## Initialize client
client = ElsClient(apikey)
client.inst_token = '' # leave it blank unless you have it

## 3. Find a unique Scopus affiliation ID for a school of interest

In [4]:
school_srch_results = affiliation_id_serch('Notre Dame', client)

print('Found ', len(school_srch_results), ' schools \n')

print('{:<15} {:>}'.format('Affiliation ID', '| Afiiliation Name'))
print('-'*40)

for school in school_srch_results:
    school_id =  school['dc:identifier'].split(':')[1]
    school_name = school['affiliation-name']
    print( '{:<15}  {:>}'.format(school_id, school_name))


Found  25  schools 

Affiliation ID  | Afiiliation Name
----------------------------------------
60021508         University of Notre Dame
60006635         University of Namur
60021478         Hospital Notre-Dame
60031487         University of Notre Dame Australia
60068771         Notre Dame University, Lebanon
60030780         Saint Marys College
60007259         Notre Dame Radiation Laboratory
60003038         Notre Dame of Maryland University
60070471         Hopital Notre Dame de Bon Secours - Centre Hospitalier Regional Metz Thionville
60028660         Indiana University School of Medicine South Bend
60026879         Hopital Notre Dame de Bonne Secours
60070491         Clinique Notre-Dame
106591450        Notre Dame Law School
60030396         Nortre Dame Seishin University
60086792         Notre Dame De Namur University
112637669        Notre Dame Law School
100331706        University of Notre Dame
60082327         Rural Clinical School of Western Australia
60070058         Clin

##### Search resulted in 25 different affiliations. First one is the one we want

In [5]:
school = school_srch_results[0]

#keep Scopus ID in the school_id variable
school_id = school['dc:identifier'].split(':')[1]
school_name = school['affiliation-name']


Here is other information we already know about this school from the search result

In [6]:
school

{'@_fa': 'true',
 'link': [{'@_fa': 'true',
   '@ref': 'self',
   '@href': 'https://api.elsevier.com/content/affiliation/affiliation_id/60021508'},
  {'@_fa': 'true',
   '@ref': 'search',
   '@href': 'https://api.elsevier.com/content/search/scopus?query=af-id%2860021508%29'},
  {'@_fa': 'true',
   '@ref': 'scopus-affiliation',
   '@href': 'https://www.scopus.com/affil/profile.uri?afid=60021508&partnerID=HzOxMe3b&origin=inward'}],
 'prism:url': 'https://api.elsevier.com/content/affiliation/affiliation_id/60021508',
 'dc:identifier': 'AFFILIATION_ID:60021508',
 'eid': '10-s2.0-60021508',
 'affiliation-name': 'University of Notre Dame',
 'name-variant': [{'@_fa': 'true', '$': 'University Of Notre Dame'}],
 'document-count': '48797',
 'city': 'Notre Dame',
 'country': 'United States',
 'parent-affiliation-id': '0'}

## 4. Find author by last and firsrt name and obtain his unique Scopus ID

In [7]:
first_name = 'Yurii'
last_name = 'Morozov'

auth_srch = ElsSearch('AUTHLASTNAME(%s)'%last_name + ' AUTHFIRST(%s)'%first_name,'author')
auth_srch.execute(client)

print ("Found ", len(auth_srch.results), " authors \n")
authorfound = auth_srch.results[0]

print('{:<6} {:<6} {:<12} {:<15} {:>}'.format('First name |', 'Last name |', 'Scopus ID |', 'Affil ID', '| Affil name'))
print('-'*80)
for author in auth_srch.results:
    #let's look on every author and print the name and affiliaiton stored in Scopus  
    author_id = author['dc:identifier'].split(':')[1]
    first_name_scopus = author['preferred-name']['given-name']
    last_name_scopus = author['preferred-name']['surname']
    affil_name = author['affiliation-current']['affiliation-name']
    affil_id = author['affiliation-current']['affiliation-id']
    
    print('{:<12} {:<11} {:<14} {:<14} {:>}'.format(first_name_scopus, last_name_scopus, author_id, affil_id, affil_name))


Found  5  authors 

First name | Last name | Scopus ID |  Affil ID        | Affil name
--------------------------------------------------------------------------------
Yu A.        Morozov     7103409464     60104831       Kotel'nikov Institute of Radio Engineering and Electronics of Russian Academy of Sciences, Saratov Branch
Yu V.        Morozov     7103409393     60026473       Engelhardt Institute of Molecular Biology, Russian Academy of Sciences
Yu N.        Morozov     7103409423     60007457       Lomonosov Moscow State University
Yurii V.     Morozov     55966498300    60021508       University of Notre Dame
Yurii V.     Morozov     55963254500    60033394       Novosibirsk State Technical University


#### We are looking for author number 4 in the above list. 
#### We can select him manulay or specify a shool name or school affiliation ID in the search query

In [8]:
last_name = 'Morozov'
first_name = 'Yurii'
school_id = '60021508'

auth_srch = ElsSearch('AUTHLASTNAME(%s)'%last_name + ' AUTHFIRST(%s)'%first_name + ' AF-ID(%s)'%school_id, 'author')
auth_srch.execute(client)

print ("Found ", len(auth_srch.results), " authors ")
authorfound = auth_srch.results[0]

for author in auth_srch.results:
    #let's look on every author and print the name and affiliaiton stored in Scopus  
    author_id = author['dc:identifier'].split(':')[1]
    first_name_scopus = author['preferred-name']['given-name']
    last_name_scopus = author['preferred-name']['surname']
    affil_name = author['affiliation-current']['affiliation-name']
    affil_id = author['affiliation-current']['affiliation-id']
    
    print('{:<12} {:<11} {:<14} {:<14} {:>}'.format(first_name_scopus, last_name_scopus, author_id, affil_id, affil_name))

Found  1  authors 
Yurii V.     Morozov     55966498300    60021508       University of Notre Dame


This looks better.  
Keep in mind however, that if author's current affiliation ID is different from the one specified in the request, search will result in error. 

#### Lets look what other information about the author we already know

In [9]:
author

{'@_fa': 'true',
 'link': [{'@_fa': 'true',
   '@ref': 'self',
   '@href': 'https://api.elsevier.com/content/author/author_id/55966498300'},
  {'@_fa': 'true',
   '@ref': 'search',
   '@href': 'https://api.elsevier.com/content/search/author?query=au-id%2855966498300%29'},
  {'@_fa': 'true',
   '@ref': 'scopus-citedby',
   '@href': 'https://www.scopus.com/author/citedby.uri?partnerID=HzOxMe3b&citedAuthorId=55966498300&origin=inward'},
  {'@_fa': 'true',
   '@ref': 'scopus-author',
   '@href': 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=55966498300&origin=inward'}],
 'prism:url': 'https://api.elsevier.com/content/author/author_id/55966498300',
 'dc:identifier': 'AUTHOR_ID:55966498300',
 'eid': '9-s2.0-55966498300',
 'preferred-name': {'surname': 'Morozov',
  'given-name': 'Yurii V.',
  'initials': 'Y.V.'},
 'name-variant': [{'@_fa': 'true',
   'surname': 'Morozov',
   'given-name': 'Yurii',
   'initials': 'Y.'}],
 'document-count': '12',
 'subject-area': [{'@abb

#### As you can see, search result dictionary contains number of publications for a given author accesible via 'document-count' key


In [10]:
print("Author", author['preferred-name']['given-name'], author['preferred-name']['surname'], "has", author['document-count'], " publications")
print("Scopus ID:", author['dc:identifier'].split(':')[1])

Author Yurii V. Morozov has 12  publications
Scopus ID: 55966498300


In [11]:
first_name = 'Yurii'
last_name = 'Morozov'
school_id = '60021508'

search_auth_by_name(first_name, last_name, school_id, client)

('success',
 {'@_fa': 'true',
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/author/author_id/55966498300'},
   {'@_fa': 'true',
    '@ref': 'search',
    '@href': 'https://api.elsevier.com/content/search/author?query=au-id%2855966498300%29'},
   {'@_fa': 'true',
    '@ref': 'scopus-citedby',
    '@href': 'https://www.scopus.com/author/citedby.uri?partnerID=HzOxMe3b&citedAuthorId=55966498300&origin=inward'},
   {'@_fa': 'true',
    '@ref': 'scopus-author',
    '@href': 'https://www.scopus.com/authid/detail.uri?partnerID=HzOxMe3b&authorId=55966498300&origin=inward'}],
  'prism:url': 'https://api.elsevier.com/content/author/author_id/55966498300',
  'dc:identifier': 'AUTHOR_ID:55966498300',
  'eid': '9-s2.0-55966498300',
  'preferred-name': {'surname': 'Morozov',
   'given-name': 'Yurii V.',
   'initials': 'Y.V.'},
  'name-variant': [{'@_fa': 'true',
    'surname': 'Morozov',
    'given-name': 'Yurii',
    'initials': 'Y.'}],
  'document-co

#### In the following example we will try to find author's Scopus ID using his last and first names and affiliation ID. 
#### And in case of an error, perform a search only with first and last name. Use *search_auth_by_name()* function defined in the beginning.

In [12]:
fstname = 'Zachary'
lstname = 'Schultz'
school_id = '60021508'

status, author  = search_auth_by_name(fstname, lstname, school_id, client)

print('provided affiliation ID ', school_id)
print('Search status =', status)
print('Current affiliation = ', author['affiliation-current']['affiliation-name'], author['affiliation-current']['affiliation-id'])

print("Author",author['preferred-name']['given-name'], author['preferred-name']['surname'], "has", author['document-count'], " publications")
print("Scopus ID:", author['dc:identifier'].split(':')[1])

provided affiliation ID  60021508
Search status = new-affil
Current affiliation =  Ohio State University 60003500
Author Zachary D. Schultz has 63  publications
Scopus ID: 6506722050


Satus 'new-affil' means that author was not found with provided affiliation id

## 5. Obtain number of publications, citation count, and h-index for a given author 

In [13]:
author_scopus_id = '55966498300'
status, search_result, npubs, ncits, hindex = auth_metrics(author_scopus_id, client)

print(search_result['author-profile']['preferred-name']['given-name'], search_result['author-profile']['preferred-name']['surname'])
print("Number of publications %i."%npubs, "Citations %i."%ncits, " h-index %i."%hindex)

Yurii V. Morozov
Number of publications 12. Citations 151.  h-index 5.


##### As perviously dictionary search_result contains more information about the author such as affiliation history, name variants, journals author has publications, etc.  
##### Below is the list of dicitonary keys

In [14]:
search_result.keys()

dict_keys(['@status', '@_fa', 'coredata', 'affiliation-current', 'affiliation-history', 'subject-areas', 'author-profile', 'h-index'])

##### Among others, I found useful 'publication-range' in 'author-profile' providing interval of years when current author published papers

In [15]:
search_result['author-profile']['publication-range']

{'@end': '2018', '@start': '2013'}

## 6. Obtain list of publications for a given author using author's Scopus ID

In [16]:
author_scopus_id = '55966498300'
publications, full_result = author_pubs(author_scopus_id, client)


Variable *publications* is a list of papers, where for every paper corresponding dictionary contains publiaction year ('year'),  
number of citation ('cited'), scopus ID of this document ('scopusid'), title ('title') and journal where the paper was published ('journal')

In [17]:
for publ in publications:
    print('')
    print(publ['year'], publ['title'])
    print( 'cited ', publ['cited'], 'times',  ' scopus ID:', publ['scopusid'])
    print('published in:', publ['journal'])


2018 A quantitative and spatially resolved analysis of the performance-bottleneck in high efficiency, planar hybrid perovskite solar cells
cited  2 times  scopus ID: 85045952698
published in: Energy and Environmental Science

2017 Rationalizing the light-induced phase separation of mixed halide organic-inorganic perovskites
cited  17 times  scopus ID: 85026885306
published in: Nature Communications

2017 Photoluminescence Up-Conversion in CsPbBr<inf>3</inf>Nanocrystals
cited  2 times  scopus ID: 85031306570
published in: ACS Energy Letters

2017 Defect-Mediated CdS Nanobelt Photoluminescence Up-Conversion
cited  3 times  scopus ID: 85026872182
published in: Journal of Physical Chemistry C

2017 Fluorescence intermittency originates from reclustering in two-dimensional organic semiconductors
cited  0 times  scopus ID: 85013631673
published in: Nature Communications

2016 Transforming Layered to Nonlayered Two-Dimensional Materials: Cation Exchange of SnS<inf>2</inf>to Cu<inf>2</inf>SnS



If you need more detailes about a publication *full_result* conatains more information

In [18]:
len(full_result), type(full_result)

(12, list)

In [19]:
full_result[0].keys()

dict_keys(['@_fa', 'link', 'prism:url', 'dc:identifier', 'eid', 'dc:title', 'dc:creator', 'prism:publicationName', 'prism:issn', 'prism:eIssn', 'prism:volume', 'prism:issueIdentifier', 'prism:pageRange', 'prism:coverDate', 'prism:coverDisplayDate', 'prism:doi', 'citedby-count', 'affiliation', 'prism:aggregationType', 'subtype', 'subtypeDescription', 'source-id', 'openaccess', 'openaccessFlag'])

#### For every publucation full_result list contains things like issn, doi, type of the journal, etc. 
#### See full output for one of the publicaitons below

In [20]:
full_result[7]

{'@_fa': 'true',
 'link': [{'@_fa': 'true',
   '@ref': 'self',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84940434926'},
  {'@_fa': 'true',
   '@ref': 'author-affiliation',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84940434926?field=author,affiliation'},
  {'@_fa': 'true',
   '@ref': 'scopus',
   '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=84940434926&origin=inward'},
  {'@_fa': 'true',
   '@ref': 'scopus-citedby',
   '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=84940434926&origin=inward'}],
 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/84940434926',
 'dc:identifier': 'SCOPUS_ID:84940434926',
 'eid': '2-s2.0-84940434926',
 'dc:title': 'Optical constants and dynamic conductivities of single layer MoS<inf>2</inf>, MoSe<inf>2</inf>, and WSe<inf>2</inf>',
 'dc:creator': 'Morozov Y.',
 'prism:publicationName': 'Applied Physics Letters',
 'prism:issn': '00036951',
 'p

### 7. Obtain detailed information about a given publication using Scopus ID

In [22]:
publication_scopus_id = '84940434926'
scp_doc = AbsDoc(scp_id = publication_scopus_id)
if scp_doc.read(client):
    print("Publication record obtained successfully")
    result = scp_doc.data
else:
    print("Something went wrong ... ")
    result = ''

Publication record obtained successfully


#### result contains a lot of information about the paper, author list and their affiliations, abstract of the publication, reference list, etc.  
below is the abstract of the given paper

In [23]:
scp_doc.data['item']['bibrecord']['head']['abstracts']#reference (['head', 'item-info', 'tail'])

'© 2015 AIP Publishing LLC. The complex optical constants of single layer MoS2, MoSe2, and WSe2 transition metal dichalcogenides (TMDCs) have been measured using concerted frequency-dependent transmittance and reflectance measurements. Absolute absorptivities as well as complex refractive indices and dielectric permittivities have been extracted. Comparisons to associated bulk responses reveal differences due to increased electron-hole interactions in single layer TMDCs. In parallel, corresponding complex optical conductivities (σ) have been determined. For MoS2, extracted σ-values qualitatively agree with recent theoretical estimates. Significant differences exist, though, between experiment and theory regarding the imaginary part of σ. In all cases, the current approach distinguishes itself to other measurements of single layer TMDC optical constants in which it does not rely on Kramers-Kronig transformations of reflectance data.'

#### Save information about this publication to json file *publication_record.json*

In [24]:
with open('publication_record.json', 'w') as fp:
    json.dump(result, fp, indent=4) 